In [ ]:
#%matplotlib inline
%matplotlib notebook
import numpy as np
import os

import strawb
import shapely.geometry
import cv2

import matplotlib.pyplot as plt

import strawb
import scipy.ndimage
import scipy.spatial
import scipy.interpolate

path = os.path.join(strawb.Config.repository_home, 'resources/image_cluster_search')

# Load images

In [ ]:
mounting_mod_dict = {
    'TUMPMTSPECTROMETER001': cv2.imread('TUMPMTSPECTROMETER001_mounting_mod.png')[:,:,::-1],
    'TUMPMTSPECTROMETER002': cv2.imread('TUMPMTSPECTROMETER002_mounting_mod.png')[:,:,::-1],
}

# gen. masked array for all devices
devices = list(mounting_mod_dict.keys())

for i in devices:
    img = np.ma.array(mounting_mod_dict[i],
                      mask=np.zeros_like(mounting_mod_dict[i])
                     )   

    # define the color threshold
    m = img[:,:,0] > 200
    m &= img[:,:,1] < 50
    m &= img[:,:,2] < 50

    img.mask |= m.reshape((*m.shape, 1))
    mounting_mod_dict[i] = img

## Plot images

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=len(mounting_mod_dict), squeeze=False, sharex='row', sharey='row')
ax = ax.flatten()

for i, dev_i in enumerate(mounting_mod_dict):
    ax[2*i].imshow(img)
    ax[2*i+1].imshow(img.filled((0,0,0)))
    
plt.tight_layout()

## Save the mask
strawb takes the mounting_mask.npz for the config

In [ ]:
# save the mask in separate file
file_name = os.path.abspath(os.path.join(strawb.Config.repository_home,
                                         'src/strawb/sensors/camera/mounting_mask.npz'))
np.savez(file_name, **{i: np.any(~mounting_mod_dict[i].mask, axis=-1) for i in mounting_mod_dict})

# for testing if written
file_test = np.load(file_name)
file_test.files

In [ ]:
def cut_fov(image, points, axis=0):
    min_xy = points.min(axis=0).astype(int)
    max_xy = points.max(axis=0).astype(int)
    slicer = (*[None]*axis, slice(min_xy[0], max_xy[0]), slice(min_xy[1], max_xy[1]))
    return image[slicer]

def equalization(image):
    # convert from RGB color-space to YCrCb
    ycrcb_img = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    if isinstance(ycrcb_img.dtype, np.float_):
        ycrcb_img = (ycrcb_img * 2**8).astype(np.uint8) 
    elif isinstance(ycrcb_img.dtype, np.uint8):
        pass
    elif isinstance(ycrcb_img.dtype, np.uint16):
        ycrcb_img = (ycrcb_img / 2).astype(np.uint8)
    
    # equalize the histogram of the Y channel
#     ycrcb_img[:, :, 0] = cv2.equalizeHist(ycrcb_img[:, :, 0])
    
    # create a CLAHE object (Arguments are optional).
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=tuple([8]*2))
    ycrcb_img[:, :, 0] = clahe.apply(ycrcb_img[:, :, 0])

    # convert back to RGB color-space from YCrCb
    return cv2.cvtColor(ycrcb_img, cv2.COLOR_YCrCb2BGR)

In [ ]:
%matplotlib notebook
ncols = 4
fig, ax = plt.subplots(ncols=ncols, nrows=len(mounting_mod_dict), squeeze=False, 
                       #sharex='row', sharey='row'
                      )
ax = ax.flatten()

c = 255//2

for i, dev_i in enumerate(mounting_mod_dict):
    cam_conf = strawb.camera.Config(dev_i)
    box = cv2.minAreaRect(np.argwhere(cam_conf.mask_mounting))
    points = cv2.boxPoints(box)
    print(dev_i, ' ', box)

    img = mounting_mod_dict[dev_i]
    ax[ncols*i].set_ylabel(dev_i.replace('TUM', ''))
    ax[ncols*i].imshow(img)
    ax[ncols*i+1].imshow(img.filled((c,c,c)))
    
    cs = ax[ncols*i+1].contourf(~img.mask.any(axis=-1), 1, hatches=['xxx', ''], alpha=1, )
    # ------------------------------
    # New bit here that handles changing the color of hatches
    colors = ['maroon', 'red', 'darkorange', 'gold', 'forestgreen',
              'darkturquoise', 'dodgerblue', 'darkviolet']
    # For each level, we set the color of its hatch 
    for j, collection in enumerate(cs.collections):
    #     collection.set_edgecolor(colors[j % len(colors)])
        collection.set_edgecolor('C0')
        collection.set_facecolor('none')#(0,0,0,0))

    # Doing this also colors in the box around each level
    # We can remove the colored line around the levels by setting the linewidth to 0
    for collection in cs.collections:
        collection.set_linewidth(1.)
    
    img_cut = cut_fov(img, points)
    ax[ncols*i+2].imshow(img_cut.filled((c,c,c)))
    ax[ncols*i+3].imshow(equalization(img_cut.filled((c,c,c))))
    
ax[0].set_title('Modified')
ax[1].set_title('masked red')
ax[2].set_title('+ cropped')
ax[3].set_title('+ equalization')
plt.tight_layout()

In [ ]:
import shapely.geometry

def get_contours(mask):
    # get contours, cv2 need uint, here uint8
    contours, hierarchy = cv2.findContours(mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours = [i.reshape((-1, 2)) for i in contours]
    return contours, hierarchy

def simplify_contours(contours, simplify_tolerance = 1):
    # simplify contours with shapely
    contours_s = []
    for i in contours:
        poly = shapely.geometry.Polygon(i)
        poly_s = poly.simplify(tolerance=simplify_tolerance)
        contours_s.append(np.array(poly_s.boundary.coords[:]))
    return contours_s

def get_contours_simplify(mask, simplify_tolerance = 1):
    # get contours, cv2 need uint, here uint8
    contours, hierarchy = get_contours(mask)
    contours = simplify_contours(contours, simplify_tolerance=simplify_tolerance)
    return contours, hierarchy

In [ ]:
# Plot all contours for all devices. There should be only one contour per device
plt.figure()
for i, dev_i in enumerate(mounting_mod_dict):
    cam_conf = strawb.camera.Config(dev_i)
    contours, hierarchy = get_contours_simplify(cam_conf.mask_mounting)

    if len(contours) != 1:
        print(f"WARNING: {dev_i} doesn't has exacly 1 contour. Got: {len(contours)}")
    for j, c_j in enumerate(contours):
        plt.plot(*c_j.T, label=f'{dev_i.replace("TUM","")}_contour_{j}')
        
plt.legend()

# [OLD] Load images with turned on LED lights

# Modifiy picture by hand to mask mounting


In [ ]:
with np.load('LED_images.npz') as f:
        images = f['images']
        
# select one bright image
image = images[-1]
        
if False:
    # Write image as png to modify it that the mounting is red ([255,0,0]) e.g. with photoshop
    # imwrite needs 16bit uint and BGR instead of RGB <-> [:,:,::-1]
    cv2.imwrite('mounting.png', (image * 2**16).astype(np.uint16)[:,:,::-1])

In [ ]:
images_list = list(images.astype(np.float32))

images_list_hdr = [images_list[1], images_list[3], images_list[-1]]
fig, ax = plt.subplots(figsize=(10,5), ncols=len(images_list_hdr), sharex=True, sharey=True, squeeze=False)
ax = ax.flatten()
for i, img_i in enumerate(images_list_hdr):
    ax[i].imshow(strawb.camera.tools.equalization(img_i))